In [ ]:
# This python script calculates the log2FC, first by finding average FPKM of a particular gene across all the tumor cases and then similarly for the normal cases. Then divides normal by tumor and takes log base 2.
#Also calculates pvalue
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

In [ ]:
output_file_name= "./Log2FC_Avg_pvalue/m_vs_normal_Log2FC_pvalue.csv"

In [ ]:
tumor_df = pd.read_csv('m_merged.csv')
normal_df = pd.read_csv('normal_merged.csv')

df = pd.DataFrame()
df['gene_id']=normal_df['gene_id']
df['gene_name']=normal_df['gene_name']
df.head()

In [ ]:
row_means_normal = normal_df.iloc[:, 2:].mean(axis=1)
df['Avg_FPKM_normal'] = row_means_normal


In [ ]:
row_means_tumor = tumor_df.iloc[:, 2:].mean(axis=1)
df['Avg_FPKM_tumor'] = row_means_tumor
df.head()

In [ ]:
df.tail()

In [ ]:
df['FC'] = df['Avg_FPKM_tumor'] / df['Avg_FPKM_normal']
df['FC'] = df['FC'].replace([np.inf, -np.inf], np.nan)

df['log2FC'] = np.log2(df['Avg_FPKM_tumor'] / df['Avg_FPKM_normal'])
df['log2FC'] = df['log2FC'].replace([np.inf, -np.inf], np.nan)

df['Regulation'] = np.where(df['log2FC'] > 2, 'Up', np.where(df['log2FC'] < -2, 'Down', '-'))

df.head()

In [ ]:

p_values = []
for i in range(len(df)):
    control = normal_df.iloc[i, 2:].to_list()
    treated = tumor_df.iloc[i, 2:].to_list()
    p_value = ttest_ind(control, treated, equal_var=True).pvalue
    p_values.append(p_value)

df['pvalue'] = p_values


In [ ]:
df.tail()

In [ ]:
df.to_csv(output_file_name, index=False)